In [1]:
import numpy as np
from scipy.spatial import distance
import random
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import pdist
import time
import json
from scipy.cluster import hierarchy

In [2]:
def merge_list(list_of_list):
    list_final = []
    for i in list_of_list:
        list_final += i    
    return list(set(list_final))


def density_calc(list_base, list_point):
    n = len(list_point)
    list_base = list(set(list_base))
    list_density =[]
    list_point_final = []
    for i in list_base:
        if i in list_point:
            list_density.append(list_point.count(i)/n)
            list_point_final.append(i)
        else:
            list_density.append(0) #changed
    return  list_point_final, list_density



def density_calc_list(list_of_list, list_base):
    list_density = []
    for i in list_of_list:
        list_density.append(np.array([density_calc(list_base, i)[1]]).transpose())
    return list_density


def create_blank_dataset_with_metadata(m):
    data = {
        'system num': [],
        'data points': [],
    }


    data[f'label'] = []
    blank_dataset = pd.DataFrame(data)
    
    return blank_dataset


def fill_dataset_with_records(dataset, records):
    for record in records:
        dataset = pd.concat([dataset, pd.DataFrame([record])], ignore_index=True)
    return dataset

def make_record(list_of_list, list_p):
    records_to_be_added = []
    for i in range(len(list_of_list)):
        records_to_be_added.append({'system num': i, 'data points': list_of_list[i], 'p':list_p[i]})
        
    return records_to_be_added

            
def condensed_creator(arr):
    m = arr.shape[0]

    # Extract upper triangle indices
    upper_triangle_indices = np.triu_indices(m, k=1)

    # Use the indices to get the upper triangle elements
    upper_triangle_elements = arr[upper_triangle_indices]

    # Convert the elements to a list if needed
    upper_triangle_list = upper_triangle_elements.tolist()

    # Print or use the resulting list as needed
    return upper_triangle_list

def plot_dendrogram(df, save_file=False):
    columns_to_filter = [str(i) for i in range(len(df))]
    df_filter = df[columns_to_filter]
    filled_df = df_filter.fillna(0)
    matrix = filled_df.values
    matrix_final = matrix + matrix.transpose()
    
    scaled_matrix = matrix_final
    np.fill_diagonal(scaled_matrix, 0)
    
    matrix_final = condensed_creator(scaled_matrix)
    linkage_matrix = linkage(matrix_final, method='complete')
    
    plt.figure(figsize=(10, 7))
    dendrogram(linkage_matrix, color_threshold=-np.inf, above_threshold_color='gray')
    plt.xlabel('Systems')  # Set the x-axis label to 'System'
    plt.xticks([])  # Remove x-axis tick labels
    plt.ylabel('Distance')
    if save_file:
        plt.savefig('../../results/plot_dendrogram.png', format='png', dpi=1000)
    plt.show()
    
    
def silhouette_score_agglomerative(df):
    columns_to_filter = [str(i) for i in range(len(df))]
    df_filter = df[columns_to_filter]
    filled_df = df_filter.fillna(0)
    matrix = filled_df.values
    matrix_final = matrix + matrix.transpose()
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    scaled_matrix = (matrix_final - min_val)
    np.fill_diagonal(scaled_matrix, 0)
    silhouette_score_list = []
    for i in range(2, len(df)):
        index_list = cluster_list_creator(df, i)
        silhouette_score_list.append(silhouette_score(scaled_matrix, index_list, metric='precomputed'))
    return silhouette_score_list

def entropy(matrix):
    matrix = np.array(matrix)
    non_zero_entries = matrix[matrix > 0]
    entropy_value = -np.sum(non_zero_entries * np.log(non_zero_entries))

    return entropy_value

def cluster_list_creator(df, num_of_clusters):
    
    columns_to_filter = [str(i) for i in range(len(df))]
    df_filter = df[columns_to_filter]
    filled_df = df_filter.fillna(0)
    matrix = filled_df.values
    matrix_final = matrix + matrix.transpose()
    
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    scaled_matrix = matrix_final
    np.fill_diagonal(scaled_matrix, 0)
    matrix_final = condensed_creator(scaled_matrix)

    linkage_matrix = linkage(matrix_final, method='complete')
    
    
    height = np.shape(linkage_matrix)[0]
    list_linkage = [[i] for i in range(len(df))]
    for i in range(height):
        list_linkage.append(list_linkage[int(linkage_matrix[i][0])] + list_linkage[int(linkage_matrix[i][1])])
        
        
    
    list_linkage_inverse = list_linkage[::-1]
    list_final = list_linkage_inverse[num_of_clusters-1:]
    list_index = []
    for i in range(len(df)):
        for j in list_final:
            if i in j:
                list_index.append(list_final.index(j))
                break

    return list_index

def calculate_OT_cost(p, q, reg, cost_matrix, num_iterations, stop_theshold):
    p = np.array([p]).T
    q = np.array([q]).T
    Xi = np.exp(-cost_matrix / reg)
    v_n = np.ones((Xi.shape[1], 1))
    v_old = v_n
    for _ in range(num_iterations):
        v_n = q / (Xi.T @ (p / (Xi @ v_n)))
        if np.linalg.norm(v_n  - v_old)<stop_theshold:
            break
        v_old = v_n
    diag_u = np.diagflat((p / (Xi @ v_n)))
    diag_v = np.diagflat(v_n)
    OT_plan = diag_u @ Xi @ diag_v
    OT_cost = np.multiply(OT_plan, cost_matrix).sum()
    return OT_plan


def fill_ot_distance(df, num_of_iterations, lambda_pen, stop_theshold):
    for i in range(len(df)):# Here we iterate among rows, and below we shall calculate the densities
        for j in range(i+1):

            cost_matrix = distance.cdist(df['data points'][i], df['data points'][j])
            min_time = time.time()
            OT_plan_test = calculate_OT_cost(df['p'][i], df['p'][j], lambda_pen, cost_matrix, num_of_iterations, stop_theshold)            
            OT_cost_test = np.multiply(OT_plan_test, cost_matrix).sum()  #yakhoda
            max_time = time.time()
            df[str(i)][j] = OT_cost_test
    

    

def normalize_tuples(list_of_lists):
    num_dimensions = len(list_of_lists[0][0])  # Get the number of dimensions from the first tuple
    
    # Extract all values for each dimension
    all_values = [[] for _ in range(num_dimensions)]
    for sublist in list_of_lists:
        for i, t in enumerate(sublist):
            for j in range(num_dimensions):
                all_values[j].append(t[j])
    
    # Compute the minimum and maximum values for each dimension
    min_values = [min(dim_values) for dim_values in all_values]
    max_values = [max(dim_values) for dim_values in all_values]
    
    # Normalize each dimension of each tuple
    normalized_list_of_lists = []
    for sublist in list_of_lists:
        normalized_sublist = []
        for t in sublist:
            normalized_t = tuple((t[j] - min_values[j]) / (max_values[j] - min_values[j]) for j in range(num_dimensions))
            normalized_sublist.append(normalized_t)
        normalized_list_of_lists.append(normalized_sublist)
    
    return normalized_list_of_lists


def list_of_lists_to_json(list_of_lists):
    json_data = {}

    for idx, lst in enumerate(list_of_lists):
        json_data[idx + 1] = {
            'id': idx + 1,
            'data points': lst
        }
    
    json_output = json.dumps(json_data, indent=4)
    
    return json_output
def json_content_to_list_of_lists(json_content):
    json_data = json.loads(json_content)
    
    list_of_lists = []
    for key in sorted(json_data.keys(), key=int):
        list_of_lists.append([tuple(item) for item in json_data[key]['data points']])
    
    return list_of_lists

In [3]:
def list_to_array(*lst):
    r""" Convert a list if in numpy format """
    lst_not_empty = [a for a in lst if len(a) > 0 and not isinstance(a, list)]

    if len(lst_not_empty) == 0:
        type_as = np.zeros(0)
    else:
        type_as = lst_not_empty[0]
    if len(lst) > 1:
        return [np.from_numpay(np.array(a), type_as=type_as)
                if isinstance(a, list) else a for a in lst]
    else:
        if isinstance(lst[0], list):
            return np.from_numpy(np.array(lst[0]), type_as=type_as)
        else:
            return lst[0]
        
        
def geometricBar(weights, alldistribT):
    """return the weighted geometric mean of distributions"""
    weights, alldistribT = list_to_array(weights, alldistribT)
    assert (len(weights) == alldistribT.shape[1])
    return np.exp(np.dot(np.log(alldistribT), weights.T))


def geometricMean(alldistribT):
    """return the  geometric mean of distributions"""
    alldistribT = list_to_array(alldistribT)
    return np.exp(np.mean(np.log(alldistribT), axis=1))


def barycenter_sinkhorn(A, M, reg, weights=None, numItermax=1000,
                        stopThr=1e-4, verbose=False, log=False, warn=True):
    A, M = list_to_array(A, M)

    if weights is None:
        weights = np.ones((A.shape[1],), dtype=A.dtype) / A.shape[1]
    else:
        assert (len(weights) == A.shape[1])

    if log:
        log_dict = {'err': []}

    K = np.exp(-M / reg)

    err = 1

    UKv = np.dot(K, (A.T / np.sum(K, axis=0)).T)
    u = (geometricMean(UKv) / UKv.T).T
    for ii in range(numItermax):

        v =  (A / np.dot(K, u))
        ps = np.tile(geometricBar(weights, UKv), (A.shape[1], 1)).T
        UKv = u * np.dot(K.T, v)
        u = ps/np.dot(K, v)
        
            
            
            
        if ii % 10 == 1:
            err = np.sum(np.std(UKv, axis=1))

            if log:
                log_dict['err'].append(err)

            if err < stopThr:
                break
            if verbose:
                if ii % 200 == 0:
                    print('{:5s}|{:12s}'.format('It.', 'Err') + '\n' + '-' * 19)
                print('{:5d}|{:8e}|'.format(ii, err))
    else:
        if warn:
            warnings.warn("Sinkhorn did not converge. You might want to "
                          "increase the number of iterations `numItermax` "
                          "or the regularization parameter `reg`.")
    if log:
        log_dict['niter'] = ii
        return geometricBar(weights, UKv), log_dict
    else:
        return geometricBar(weights, UKv)
    
    
def calc_barycenter(df, cost_matrix, lambda_value):
    list_p_cluster =  [i for i in df['p']]
    matrix_p = np.column_stack(list_p_cluster)
    return barycenter_sinkhorn(matrix_p, cost_matrix, lambda_value, weights=None, numItermax=200, stopThr=0.0001, verbose=False, log=False, warn=True)


def dataframe_to_json(df, columns=None):
    """
    Convert a pandas DataFrame to a JSON format where each row index maps to a dictionary 
    of column names and their corresponding values.

    Parameters:
    df (pandas.DataFrame): The DataFrame to convert.

    Returns:
    str: JSON string representing the DataFrame.
    """
    # Convert DataFrame to dictionary format
    if columns==None:
        df_dict = df.to_dict(orient='index')
    else:
        df_dict = df[columns].to_dict(orient='index')
    for row in df_dict.values():
        for key, value in row.items():
            if isinstance(value, np.ndarray):
                row[key] = value.tolist()
    # Convert dictionary to JSON
    json_result = json.dumps(df_dict, indent=4)
    
    return json_result

In [4]:
def proximal_mapping(a, gradient, t0_beta):
    # Proximal mapping using Kullback-Leibler divergence as the Bregman divergence


    a_tilde = a * np.exp(-t0_beta * gradient)
    a_tilde /= np.sum(a_tilde)
    return a_tilde



def opt_a(X, Y_list, b_list, t0, tol=1e-9, max_iter=1000):
    n = X.shape[0]
    N = len(Y_list)
    
    # Form all n x mi matrices Mi
    M_list = [np.linalg.norm(X[:, np.newaxis] - Y, axis=2) for Y in Y_list]
#     Initialize a_hat and a_tilde
    a_hat  = (np.ones(n) / n).reshape((n, 1))
    a_tilde = a_hat
    t = 3
    converged = False
    while not converged and t < max_iter:
        beta = (t + 1) / 2
        a = (1 - beta**-1) * a_hat + beta**-1 * a_tilde
        # Form subgradient alpha
        alpha_list = [calculate_OT_cost(a, b_list[i], 0.2,M_list[i], num_iterations=100, stop_theshold=10**-9)[1] for i in range(len(b_list))]
        alpha = np.mean(-np.log(alpha_list)*0.2, axis=0)
        # Update a_tilde using the proximal mapping
        t0_beta = t0 * beta
        a_tilde = proximal_mapping(a, alpha, t0_beta)
#         # Update a_hat
        a_hat = (1 - beta**-1) * a_hat + (beta**-1) * a_tilde
        
#         # Check convergence
        if np.linalg.norm(a_tilde - a_hat) < tol:
            converged = True
        
        t += 1
    return a_hat


def find_barycenter(X, Y_list, b_list, t0, theta, tol=1e-9, max_iter=1000):
    iter_num = 1
    while iter_num< max_iter:
        n = X.shape[0]
        N = len(Y_list)
        M_list = [np.linalg.norm(X[:, np.newaxis] - Y, axis=2) for Y in Y_list]
        
        
        a_update = opt_a(X, Y_list, b_list, t0, tol=1e-2, max_iter=30)[0]
        T_list = [calculate_OT_cost(a_update, b_list[i], 0.2, M_list[i], num_iterations=50, stop_theshold=10**-4)[0] for i in range(len(b_list))]
        YT_list = [T_list[i] @ Y_list[i] for i in range(len(Y_list))]

        YT_ave = np.mean(YT_list, axis=0)
        X_old = X
        X = (1-theta) * X + theta * (np.diag((a_update**-1).T[0]) @ YT_ave)

        if np.linalg.norm(X - X_old) < tol:
            print('BROKE!'*5)
            return X, a_update
        iter_num += 1 

    return X, a_update

def calculate_OT_cost(p, q, reg, cost_matrix, num_iterations, stop_theshold):

    p = np.array([p]).T
    q = np.array([q]).T

    Xi = np.exp(-cost_matrix / reg)
    v_n = np.ones((Xi.shape[1], 1))
    v_old = v_n
    for _ in range(num_iterations):
        v_n = q / (Xi.T @ (p / (Xi @ v_n)))
        if np.linalg.norm(v_n  - v_old)<stop_theshold:
            break
        v_old = v_n
    diag_u = np.diagflat((p / (Xi @ v_n)))
    diag_v = np.diagflat(v_n)
    OT_plan = diag_u @ Xi @ diag_v
    OT_cost = np.multiply(OT_plan, cost_matrix).sum()
    return OT_plan, p / (Xi @ v_n), v_n


In [5]:
def fill_ot_distance(df, df_clusters, num_of_iterations, lambda_pen, stop_theshold):
#     print('entered fill_ot_distance')
    for i in range(len(df)):
        for j in range(len(df_clusters)):
            cost_matrix = distance.cdist(df['data points'][i], df_clusters['data points'][j])
            OT_plan_test = calculate_OT_cost(df['p'][i], df_clusters['p'][j], lambda_pen, cost_matrix, num_of_iterations, stop_theshold)[0]   
            
            cost_matrix1 = distance.cdist(df['data points'][i], df['data points'][i])
            OT_plan_test1 = calculate_OT_cost(df['p'][i], df['p'][i], lambda_pen, cost_matrix1, num_of_iterations, stop_theshold)[0]   
   
            cost_matrix2 = distance.cdist(df_clusters['data points'][j], df_clusters['data points'][j])
            OT_plan_test2 = calculate_OT_cost(df_clusters['p'][j], df_clusters['p'][j], lambda_pen, cost_matrix1, num_of_iterations, stop_theshold)[0]   
            
            OT_cost_test1 = np.multiply(OT_plan_test1, cost_matrix1).sum() 
            OT_cost_test2 = np.multiply(OT_plan_test2, cost_matrix2).sum()
            
            OT_cost_test = np.multiply(OT_plan_test, cost_matrix).sum() - 0.5*(OT_cost_test1 + OT_cost_test2)   #yakhoda
#             OT_cost_test = np.multiply(OT_plan_test, cost_matrix).sum()
            df[str(j)][i] = OT_cost_test

In [6]:
def normalize_tuples(list_of_lists):
    num_dimensions = 2  # Get the number of dimensions from the first tuple
    
    # Extract all values for each dimension
    all_values = [[] for _ in range(num_dimensions)]
    for sublist in list_of_lists:
        for i, t in enumerate(sublist):
            for j in range(num_dimensions):
                all_values[j].append(t[j])
    
    # Compute the minimum and maximum values for each dimension
    min_values = [min(dim_values) for dim_values in all_values]
    max_values = [max(dim_values) for dim_values in all_values]
#     print(min_values)
#     print(max_values)
    # Normalize each dimension of each tuple
    normalized_list_of_lists = []
    for sublist in list_of_lists:
        normalized_sublist = []
        for t in sublist:
            normalized_t = tuple((t[j] - min_values[j]) / (max_values[j] - min_values[j]) for j in range(num_dimensions))
            normalized_sublist.append(normalized_t)
        normalized_list_of_lists.append(normalized_sublist)
    
    return normalized_list_of_lists, np.array(min_values), np.array(max_values)

In [7]:
with open('../../data/tuples_list.txt', 'r') as f:
    # Read lines from the file and parse tuples of floats
    list_sim_outputs_raw = [eval(line.strip()) for line in f]

json_result = list_of_lists_to_json(list_sim_outputs_raw)
# min_time = time.time()
# for m in range(2, 20):
#     cluster_distributions_kmeans(json_result, reg=0.2, n_clusters=m, stop_theshold=10**-9, num_of_iterations=1000)
# max_time = time.time()
# print(max_time - min_time)

In [8]:
def denormalize(matrix, min_values, max_values):
#     matrix = np.array(matrix)
#     matrix = matrix * (max_values - min_values) + min_values
#     matrix = [tuple(row) for row in matrix]
    return matrix * (max_values - min_values) + min_values

In [9]:

def cluster_distributions_kmeans(dist_file,  reg=0.2, n_clusters=2, stop_theshold=10**-9, num_of_iterations=100, plt_dendrogram=True):


    list_sim_outputs_raw = json_content_to_list_of_lists(dist_file) 
    list_base = merge_list(list_sim_outputs_raw)
    list_sim_outputs = []
    p_list = []
    for i in list_sim_outputs_raw:
        list_sim_outputs.append(density_calc(i, i)[0])
        p_list.append(density_calc(i, i)[1])
    print(len(list_sim_outputs_raw))
    normalized_list_sim_outputs = normalize_tuples(list_sim_outputs)[0]
    min_values_global = normalize_tuples(list_sim_outputs)[1]
    max_values_global = normalize_tuples(list_sim_outputs)[2]

    m = len(normalized_list_sim_outputs)
    blank_df = create_blank_dataset_with_metadata(m)
    df = fill_dataset_with_records(blank_df, make_record(normalized_list_sim_outputs, p_list))
    # Display the filled dataset
    df['data points real'] = list_sim_outputs

    #Define initial centroids from the instances
    initial_clusters = random.sample(range(len(list_sim_outputs_raw)), n_clusters)
    print(f'initial_clusters is {initial_clusters}')
    #make the centroids dataset
    blank_df_clusters = create_blank_dataset_with_metadata(n_clusters)
    records_to_be_added =[]
    for i in initial_clusters:
        records_to_be_added.append({'cluster num': initial_clusters.index(i), 'p':df['p'][i], 'data points':df['data points'][i]})
    df_clusters = fill_dataset_with_records(blank_df_clusters, records_to_be_added)  

    for i in range(n_clusters):
        df[f'{i}'] = [0 for i in range(len(list_sim_outputs_raw))]
    fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)

    columns_to_consider = [str(i) for i in range(n_clusters)]
    df['label'] = df[columns_to_consider].idxmin(axis=1)
    df['label'] = df['label'].astype(int)

    cluster_itration = 5
    X = np.random.rand(20, 2)

    for _ in range(cluster_itration):
        min_values_all = []
        max_values_all = []
        list_bary_X = []
        list_bary_prob = []
        list_inputs_cluster = []
        list_p_cluster_new = []
        list_sup_cluster_new = []
        list_sup_cluster_real_new = []

        for i in range(len(df_clusters)):
 
            df_test  = df[df['label']==i]
            if len(df_test) == 1:

                df_clusters['data points'][i] = df_test['data points'].tolist()[0]
                df_clusters['p'][i] = df_test['p'].tolist()[0]

            else:
                list_column = df_test['data points']

                list_sim_outputs_cluster = list_column.tolist()



                list_sim_outputs_cluster =  list_sim_outputs_cluster

                list_base_cluster = merge_list(list_sim_outputs_cluster)
                list_of_arrays = [np.array(inner_list) for inner_list in list_sim_outputs_cluster]


                b_list = [(np.ones(20) / 20 ).tolist() for _ in range(len(list_sim_outputs_cluster))]
                t0 = 0.1
                theta = 0.1
                reg = 0.2
                bary_X, bary_a = find_barycenter(X, list_of_arrays, b_list, t0, theta, tol=1e-6, max_iter=50)
                bary_X = [tuple(row) for row in bary_X]



                df_clusters['data points'][i] = bary_X
                df_clusters['p'][i] = bary_a.T.tolist()[0]
        fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)
        columns_to_consider = [str(i) for i in range(n_clusters)]
        old_label = df['label'].tolist()
        list_indices = df[columns_to_consider].idxmin(axis=1).astype(int).tolist()
#         print(list(set(list_indices)))
        if len(list(set(list_indices))) < n_clusters:
            print('ONE IS MISSING')
            return df
            missing_indice =[]
            for k in range(n_clusters):
                if k not in list(set(list_indices)):
                    missing_indice.append(k)
            for k in missing_indice:
                
                list_indices[k] = k # you could make it variable
            
#         df['label'] = df[columns_to_consider].idxmin(axis=1).astype(int).tolist()
        df['label'] = list_indices
#         print(df[columns_to_consider].idxmin(axis=1).astype(int).tolist())
#         df['label'] = df['label'].astype(int) 
        print(df['label'].tolist())
        #what if one of the labels is gone?
#         if len(df['label'].tolist())
        if old_label == df['label'].tolist():
            return df

In [10]:

def cluster_distributions_kmeans(dist_file,  reg=0.2, n_clusters=2, stop_theshold=10**-9, num_of_iterations=100, plt_dendrogram=True):


    list_sim_outputs_raw = json_content_to_list_of_lists(dist_file) 
    support_size = len(list_sim_outputs_raw[0])
    list_base = merge_list(list_sim_outputs_raw)
    list_sim_outputs = []
    p_list = []
    for i in list_sim_outputs_raw:
        list_sim_outputs.append(density_calc(i, i)[0])
        p_list.append(density_calc(i, i)[1])
    normalized_list_sim_outputs = normalize_tuples(list_sim_outputs)[0]
    min_values_global = normalize_tuples(list_sim_outputs)[1]
    max_values_global = normalize_tuples(list_sim_outputs)[2]

    m = len(normalized_list_sim_outputs)
    blank_df = create_blank_dataset_with_metadata(m)
    df = fill_dataset_with_records(blank_df, make_record(normalized_list_sim_outputs, p_list))
    # Display the filled dataset
    df['data points real'] = list_sim_outputs

    #Define initial centroids from the instances
    initial_clusters = random.sample(range(len(list_sim_outputs_raw)), n_clusters)
    #make the centroids dataset
    blank_df_clusters = create_blank_dataset_with_metadata(n_clusters)

    records_to_be_added =[]
    for i in initial_clusters:
        records_to_be_added.append({'cluster num': initial_clusters.index(i), 'p':df['p'][i], 'data points':df['data points'][i]})
    df_clusters = fill_dataset_with_records(blank_df_clusters, records_to_be_added)  

    for i in range(n_clusters):
        df[f'{i}'] = [0 for i in range(len(list_sim_outputs_raw))]
    fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)

    columns_to_consider = [str(i) for i in range(n_clusters)]
    df['label'] = df[columns_to_consider].idxmin(axis=1)
    df['label'] = df['label'].astype(int)
    list_indices = df[columns_to_consider].idxmin(axis=1).astype(int).tolist()
    if len(list(set(list_indices))) < n_clusters:
        return df
    cluster_itration = support_size
    X = np.random.rand(support_size, 2)

    for _ in range(cluster_itration):
        min_values_all = []
        max_values_all = []
        list_bary_X = []
        list_bary_prob = []
        list_inputs_cluster = []
        list_p_cluster_new = []
        list_sup_cluster_new = []
        list_sup_cluster_real_new = []

        for i in range(len(df_clusters)):

            df_test  = df[df['label']==i]
            if len(df_test) == 1:

                df_clusters['data points'][i] = df_test['data points'].tolist()[0]
                df_clusters['p'][i] = df_test['p'].tolist()[0]

            else:
                list_column = df_test['data points real']

                list_sim_outputs_cluster = list_column.tolist()


                min_values_all= normalize_tuples(list_sim_outputs_cluster)[1]
                max_values_all= normalize_tuples(list_sim_outputs_cluster)[2]
                list_sim_outputs_cluster =  normalize_tuples(list_sim_outputs_cluster)[0]

                list_base_cluster = merge_list(list_sim_outputs_cluster)
                list_of_arrays = [np.array(inner_list) for inner_list in list_sim_outputs_cluster]


                b_list = [(np.ones(support_size) / support_size ).tolist() for _ in range(len(list_sim_outputs_cluster))]
                t0 = 0.01
                theta = 0.05
                reg = 0.2
                bary_X, bary_a = find_barycenter(X, list_of_arrays, b_list, t0, theta, tol=1e-6, max_iter=200)
                bary_X = [tuple(row) for row in bary_X]
                bary_X = bary_X * (max_values_all - min_values_all) + min_values_all 
                list_bary_X.append(bary_X)
                list_bary_prob.append(bary_a.T.tolist()[0])

                df_clusters['data points'][i] = (bary_X - min_values_global) / (max_values_global - min_values_global)
                df_clusters['p'][i] = bary_a.T.tolist()[0]
        fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)
        columns_to_consider = [str(i) for i in range(n_clusters)]
        old_label = df['label'].tolist()
        list_indices = df[columns_to_consider].idxmin(axis=1).astype(int).tolist()

        if len(list(set(list_indices))) < n_clusters:

            return df
            

        df['label'] = list_indices

        if old_label == df['label'].tolist():
            return df

In [12]:
with open('../../data/tuples_list.txt', 'r') as f:
    # Read lines from the file and parse tuples of floats
    list_sim_outputs_raw = [eval(line.strip()) for line in f]

json_result = list_of_lists_to_json(list_sim_outputs_raw)

min_time = time.time()
cluster_distributions_kmeans(json_result, reg=0.2, n_clusters=4, stop_theshold=10**-9, num_of_iterations=500)


In [ ]:
with open('../../data/tuples_list.txt', 'r') as f:
    # Read lines from the file and parse tuples of floats
    list_sim_outputs_raw = [eval(line.strip()) for line in f]

json_result = list_of_lists_to_json(list_sim_outputs_raw)
min_time = time.time()
for m in range(2, 10):
    cluster_distributions_kmeans(json_result, reg=0.2, n_clusters=m, stop_theshold=10**-9, num_of_iterations=500)
max_time = time.time()

In [ ]:
kmeans_time = []
for i in range(1, 11):
    filename = f'../../data/tuples_list_{i}.txt'
    print(f"Processing {filename}")
    
    with open(filename, 'r') as f:
        # Read lines from the file and parse tuples of floats
        list_sim_outputs_raw = [eval(line.strip()) for line in f]

    json_result = list_of_lists_to_json(list_sim_outputs_raw)
    
    min_time = time.time()
    for m in range(2, 10):
        cluster_distributions_kmeans(json_result, reg=0.5, n_clusters=m, stop_theshold=10**-9, num_of_iterations=500)
    max_time = time.time()
    kmeans_time.append(max_time - min_time)




In [ ]:
reg=0.2
n_clusters=2 
calculate_barycenter=False 
stop_theshold=10**-9
num_of_iterations=1000



json_result = list_of_lists_to_json(list_sim_outputs_raw)
list_sim_outputs_raw = json_content_to_list_of_lists(json_result)
list_base = merge_list(list_sim_outputs_raw)
list_sim_outputs = []
p_list = []
for i in list_sim_outputs_raw:
    list_sim_outputs.append(density_calc(i, i)[0])
    p_list.append(density_calc(i, i)[1])
print(len(list_sim_outputs_raw))
normalized_list_sim_outputs = normalize_tuples(list_sim_outputs)[0]
min_values_global = normalize_tuples(list_sim_outputs)[1]
max_values_global = normalize_tuples(list_sim_outputs)[2]



m = len(normalized_list_sim_outputs)
blank_df = create_blank_dataset_with_metadata(m)
df = fill_dataset_with_records(blank_df, make_record(normalized_list_sim_outputs, p_list))
# Display the filled dataset
df['data points real'] = list_sim_outputs

#     print(df)

#Define initial centroids from the instances
initial_clusters = random.sample(range(len(list_sim_outputs_raw)), n_clusters)


#make the centroids dataset
blank_df_clusters = create_blank_dataset_with_metadata(n_clusters)
records_to_be_added =[]
for i in initial_clusters:
#         print(i)
    records_to_be_added.append({'cluster num': initial_clusters.index(i), 'p':df['p'][i], 'data points':df['data points'][i]})
df_clusters = fill_dataset_with_records(blank_df_clusters, records_to_be_added)  

for i in range(n_clusters):
    df[f'{i}'] = [0 for i in range(len(list_sim_outputs_raw))]
fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)

columns_to_consider = [str(i) for i in range(n_clusters)]
df['label'] = df[columns_to_consider].idxmin(axis=1)
df['label'] = df['label'].astype(int)

#     print(df) 
#     print(f" labe is \n{df['label'].tolist()}")
cluster_itration = 20
X = np.random.rand(10, 2)

for _ in range(cluster_itration):
    min_values_all = []
    max_values_all = []
    list_bary_X = []
    list_bary_prob = []
    list_inputs_cluster = []
    list_p_cluster_new = []
    list_sup_cluster_new = []
    list_sup_cluster_real_new = []

    for i in range(len(df_clusters)):
        print(f'i is {i}')
        df_test  = df[df['label']==i]
        if len(df_test) == 1:

            df_clusters['data points'][i] = df_test['data points'].tolist()[0]
            df_clusters['p'][i] = df_test['p'].tolist()[0]
            
        else:
            list_column = df_test['data points real']

            list_sim_outputs_cluster = list_column.tolist()


            min_values_all= normalize_tuples(list_sim_outputs_cluster)[1]
            max_values_all= normalize_tuples(list_sim_outputs_cluster)[2]
            list_sim_outputs_cluster =  normalize_tuples(list_sim_outputs_cluster)[0]

            list_base_cluster = merge_list(list_sim_outputs_cluster)
            list_of_arrays = [np.array(inner_list) for inner_list in list_sim_outputs_cluster]

            
            b_list = [(np.ones(10) / 10 ).tolist() for _ in range(len(list_sim_outputs_cluster))]
            t0 = 0.005
            theta = 0.005
            reg = 0.2

            bary_X, bary_a = find_barycenter(X, list_of_arrays, b_list, t0, theta, tol=1e-10, max_iter=300)
            bary_X = [tuple(row) for row in bary_X]

            bary_X = bary_X * (max_values_all - min_values_all) + min_values_all 
            list_bary_X.append(bary_X)
            list_bary_prob.append(bary_a.T.tolist()[0])

            df_clusters['data points'][i] = (bary_X - min_values_global) / (max_values_global - min_values_global)
            df_clusters['p'][i] = bary_a.T.tolist()[0]

    fill_ot_distance(df, df_clusters, num_of_iterations, reg, stop_theshold)

    columns_to_consider = [str(i) for i in range(n_clusters)]
    old_label = df['label'].tolist()
    df['label'] = df[columns_to_consider].idxmin(axis=1)
    df['label'] = df['label'].astype(int) 


    if old_label == df['label'].tolist():
        print('DONE')
        print('*'*50)
        return df